In [2]:
import pickle
import numpy as np
import pandas as pd

In [6]:
f = open('../results/simpleLinear_cubedFeatures_noYClipping_again', 'rb')
res = pickle.load(f)

In [11]:
res[2]

array([ -9.74920814, -19.27806141,  -6.71612096,   1.23716825,
        -9.45237174,  -9.0095436 ,   0.1311377 ,  -4.61295311,
         2.90581709,   0.80183601,   3.22805319,  -8.71192061,
        -5.40364609,   8.98075427,   3.6148165 ,  13.41929203,
         4.36945907,   6.96467801,   5.81297467,   5.0196314 ,
         8.68775597,   3.74756416,   3.2350929 ,  -5.79391737,
        23.90890198,  46.33565489,  18.56623094,  -0.68502374,
        14.91682672,  23.2023632 ,   0.181332  ,  12.59054103,
        -4.66684105, -12.91434082,   7.94119726,   1.82575428,
         8.56092716, -28.46512801,  -7.43266581, -39.58267064,
       -10.12171325, -10.3305816 ,  -8.22786783,  -7.84614514,
       -21.35024399,  18.63224307,   2.87404541, -12.83558643,
       -13.71109163, -23.51834479, -11.31135861,  -1.31765225,
        -4.24668504, -11.25377724,  -1.45634331,  -7.97620383,
         2.48090347,  14.77835847,  -2.55407909,   8.77004725,
        -3.04100884,  18.72445416,   3.50998303,  20.96

In [12]:
def engineerdFeatures(mat):
    M = mat
    M = np.append(mat, np.power(mat,2), axis=1)
    M = np.append(M, np.power(mat,3), axis=1)
    return M

In [19]:
df = pd.read_csv('../data/train.csv', header=0)
#the total of 24 valueable features
#the first 1 is not helpful 
#however and datetime data can be used, <----tried various experiment, doesn't really help
c = len(df.iloc[0,:])
df_ = df.iloc[:,2:c]
idldata = df_.iloc[:,0:-1].values
idldata = engineerdFeatures(idldata)
tunedMean = np.mean(idldata,axis=0)
tunedStd = np.std(idldata, axis=0)

In [20]:
def get_feature_matrix(file_path):
    df = pd.read_csv(file_path, header=0)
    #the total of 24 valueable features
    #the first 1 is not helpful 
    #however and datetime data can be used, <----tried various experiment, doesn't really help
    c = len(df.iloc[0,:])
    df_ = df.iloc[:,2:c]

    if len(df_.iloc[0,:])==25:
        print('loading training data...')
        res = df_.iloc[:,0:-1].values
        res = engineerdFeatures(res)
        res = normalizeMydata(res, np.mean(res, axis=0), np.std(res, axis=0))
    else:
        print('loading testing data...')
        res = df_.values
        res = engineerdFeatures(res)
        res = normalizeMydata(res, tunedMean, tunedStd)    
        return res

In [22]:
def normalizeMydata(data, tunedMean, tunedStdd):
    data_ = np.array(data, dtype=np.float32)
    data_ = (data-tunedMean)/tunedStdd
    return data_


In [23]:
testX = get_feature_matrix('../data/test_features.csv')

loading testing data...


In [26]:
test_X_1 = np.append(testX, np.ones([len(testX[:,0]),1]), axis=1)

In [27]:
predicted_Y = np.dot(test_X_1, res[2])

In [28]:
predicted_Y


array([  90.67511866,   98.3373062 ,  141.76855464, ...,   73.76474966,
        115.3428889 ,  133.34833346])

In [29]:
file_name = 'simpleLinear_cubedFeatures_noYClipping_again'
n_res = len(predicted_Y)
index = range(1,n_res+1)
resdf = pd.DataFrame()
resdf['id'] = index
resdf['output'] = predicted_Y
resdf.to_csv('../results/'+file_name+'.csv', header = 1, index=0)